Accomodation Data Formatter 

Caleb Haynes 2/3/2020 for Johanna Jarcho 

This converts data output by a delayed discounting task presented in E-Prime. 

Conversion reformatting for input to hBayesDM R package - dd_hyperbolic.

https://rdrr.io/cran/hBayesDM/man/dd_hyperbolic.html

Requirements:

subjID
A unique identifier for each subject in the data-set.

delay_later
An integer representing the delayed days for the later option (e.g. 1, 6, 28).

amount_later
A floating point number representing the amount for the later option (e.g. 10.5, 13.4, 30.9).

delay_sooner
An integer representing the delayed days for the sooner option (e.g. 0).

amount_sooner
A floating point number representing the amount for the sooner option (e.g. 10).

choice
If amount_later was selected, choice == 1; else if amount_sooner was selected, choice == 0.


In [2]:
import pandas as pd
import numpy as np
import glob

In [11]:
'''
find all converted .edat2 -> xlsx in data directory
'''
accom_dfs = [pd.read_excel(i) for i in glob.glob("data/s*_1/DD_*.xlsx")]

In [8]:
for df in accom_dfs:
    subno = df['Subject'][1]
    #drop practice rows
    df = df.drop(df.index[0:2])
    
    '''
    renaming columns to match R package standards. 
    DelR and AmtR always have larger values, variable/column LeftRight
    determines if it will be placed on Left or Right side of screen for
    counterbalancing. Higher value is also always 'delayed' option, so 
    we can rename those columns to fit standard. Opposite for smaller 
    values which will always be in DelL and AmtL.
    '''
    df = df.rename(columns={'Subject': 'subjID', 
                            'DelR': 'delay_later',
                            'DelL': 'delay_sooner',
                            'AmtR': 'amount_later',
                            'AmtL': 'amount_sooner'})
    
    '''
    Delay Soon is always 'now,' so we need to change all to 0
    '''
    df = df.replace(df[['delay_sooner']], 0)
    
    '''
    Choice is more complex- 
    If LeftRight == 0, then Amount14 was shown on the right side of the screen, 
    if Amount14.RESP=='j'they chose what is on the right side of the screen.

    If LeftRight == 1, then Amount6 was shown on the right side of the screen, 
    f Amount6.RESP =='j, they chose the delayed reward, which was what appeared on the right side of the screen.  
    
    '''
    def label_displayed(row):
        if row['LeftRight'] == 0 and row['Amount14.RESP'] == 'j':
            return 1
        else:
            return 0
    df['choice'] = df.apply(lambda row: label_displayed(row), axis=1)
    
    hbayes_df = df[['subjID','delay_later','amount_later','delay_sooner','amount_sooner','choice']]
    
    #write df to tsv
    hbayes_df.to_csv('data/s{}_1/DD_{}.tsv'.format(subno, subno), index = False, sep = '\t')

In [24]:
'''
Now I need to join all output tsvs from above into one merged 
'''
accom_dfs = [pd.read_csv(i, sep = '\t') for i in glob.glob("data/s*_1/DD_*.tsv")]
merged = pd.concat(accom_dfs)

In [30]:
merged.to_csv('accom_merged.txt', index=False, sep = '\t')